In [ ]:
using Distributed, ClusterManagers

In [ ]:
addprocs(SlurmManager(28), partition="gen", t="00:10:00") # ask for 28 CPUs in the gen partition for 10 minutes

Run the UNIX command `hostname` on each worker Julia has spun up, so that we can see which node(s) we have acquired.

In [ ]:
@parallel for i=1:5
    run(`hostname`)
end

Create some data which we will work on. For now, this data is only accessible on the driver worker - worker 0.

In [ ]:
m  = 3_000
k  = 2_000
n  = 1_000

As = [rand(m, k) for i in 1:nworkers()]
Bs = [rand(k, n) for i in 1:nworkers()]

Cs = pmap((x,y)->x*y, As, Bs)

That's a pretty simple example. But `pmap` can do more interesting things:

In [ ]:
ms = rand(1_000:2_000, 2, div(nworkers(), 2))
ks = rand(1_000:2_000, 2, div(nworkers(), 2))
ns = rand(1_000:2_000, 2, div(nworkers(), 2))

Cs = pmap((x, y, z) -> rand(x, y)*rand(y, z), ms, ks, ns)
@show size(Cs)

@show ms
@show ns
@show size.(Cs)

Notice how `pmap` has retained the original layout of the inputs, so that the output corresponding to `m[1,2], k[1,2], n[1,2]` is placed in `Cs[1,2]`, no matter which worker it was computed on.